In [0]:
grading = False

In [0]:
from collections import defaultdict
import math
import pandas as pd

In [0]:
test_df = pd.read_csv("a7_test.csv")
if not grading:
  test_df

train_df = pd.read_csv("a7_train.csv")
if not grading:
  train_df

In [0]:
def calculate_priors(df):
  """
  TASK 1a)

  Calculates the probability of given classes, based on how often they appear
  in a DataFrame.

  You can assume that the class values for each row are in a column called
  "class".

  !!! Your function must work for any number of classes, so don't assume that 
  the number of classes is always exactly 2! !!!
  
  Return a dictionary mapping the class to its probability.
  Example: The class "apple" appears in two rows, the class "banana" in three.
  The probability for the "apple" class is 2/5, the probability for banana is 
  3/5.
  You have to return a dictionary:
    {"apple": 0.4, "banana": 0.6}

  HINT: You can use .value_counts() to count the values in a column.
    You have to select the correct column first.
  HINT: You can use .apply to modify values with the use of a function, such as
    a lambda function.

  :param df: A pandas.DataFrame containing a column "class" and a column "text". 
  :return: A dictionary mapping the class to its probability.

  """

  prior = df['class'].value_counts()
  
  prior = prior.apply(lambda x: x / prior.sum())

  return prior.to_dict()

In [38]:
############### EXTRA OUTPUT / TESTING ##################
if not grading:
  d = {"class": ["apple" if x < 3000 else "banana" for x in range(10000)], "text": ["FOO!" for x in range(10000)]}
  test = pd.DataFrame(d)
  p = calculate_priors(test)
  # correct output: {'banana': 0.7, 'apple': 0.3}
  print(p)
  p = calculate_priors(train_df)
  # correct output: {'pos': 0.5, 'neg': 0.5}
  print(p)

{'banana': 0.7, 'apple': 0.3}
{'neg': 0.5, 'pos': 0.5}


In [0]:
if not grading:
  priors = calculate_priors(train_df)

In [0]:
def calculate_class_term_frequency(df, classes):
  """
  TASK 1b)

  For each, calculates the frequency of terms appearing in texts of this class.

  In the DataFrame, each row has an entry for "text" and one for "class".
  The values in the "text" column are strings (= actual texts).
  Your task is to calculate the frequency (= how often a word appears) for each
  of the classes.

  Example: 
    There are two texts with the "apple" class. 
    There are three texts with the "banana" class.
    The word "fruit" appears five times in the texts of the "apple" class, and 
    two times for texts of the "banana" class.
    The total frequency of the word "fruit" is 7. The frequency for the "apple"
    class is 5. The frequency for the "banana" class is 2.

  To get the individual terms you can use .split(" ") (splitting by whitespace).
  Convert the terms to lowercase.

  HINT: You can iterate over the DataFrame rows with .iterrows():
  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html

  :param df: A pandas.DataFrame containing a column "class" and a column "text".
  :param classes: A list of strings containing the possible values for the "class" column.
  :return: A set containing all the terms (list of string) and a dictionary containing
  the frequency of terms appearing in texts of each class in the following format:
    {class(string): {term(string): frequency(int), ...}, ...}
  
  """



  terms = set()
  for index, row in df.iterrows():
    for word in row['text'].split(" "):
      terms.add(word.lower())

  dct = dict()
  for class_ in classes:
    dct[class_] = defaultdict(int)
    for index, row in df.iterrows():
      if row['class'] == class_:
        for term in row['text'].lower().split(" "):
          dct[class_][term] += 1







  return terms, dct

In [42]:
############### EXTRA OUTPUT / TESTING ##################
if not grading:
  terms, freqs_per_class = calculate_class_term_frequency(train_df, ["pos", "neg"])
  # correct output: 252192
  print(len(terms))
  # correct output: 5335
  print(freqs_per_class["neg"]["bad"])
  # correct output: 10
  print(freqs_per_class["pos"]["rainbow"])

252192
5335
10


In [0]:
if not grading:
  classes = priors.keys()
  terms, freqs_per_class = calculate_class_term_frequency(train_df, classes)

**Conditional Probabilities (with add-1 smoothing)**: $P(w|c) = \frac{count(w, c) + 1}{count(c) + |V|}$

Example: $P(fruit|apple) = 0.3$

$count(w, c)$ = How often word $w$ appears in documents of class $c$.

$count(c)$ = How many words appear in documents of class $c$.

$|V|$ = How many different words exist in the dataset.

**Prior**: $P_{prior}(c) = \frac{N_c}{N}$, where $N$ is the total number of documents and $N_c$ is the number of documents with class $c$

**Choosing a class for document $d$**: $argmax_{c \in C} P(c|d)$ with $P(c|d) = P_{prior}(c) * \prod_{w \in d} P(w|c)$

You can also refer to the lecture slides where you can find an example calculation.


In [44]:
if not grading:
  example_dct = {"fruit": {"apple": 0.3, "banana": 0.4}, "tree": {"apple": 0.5, "banana": 0.1}}
  print(example_dct["fruit"])  # prints the inner dictionary
  # prints the probability of "fruit" appearing in a document if the document has the class "apple"
  print(example_dct["fruit"]["apple"])

{'apple': 0.3, 'banana': 0.4}
0.3


In [0]:
def calculate_class_term_probs(terms, freqs_per_class):
  """
  TASK 1c)

  Calculates the probability that a term appears in documents of a given class 
  and returns a nested dictionary containing the probabilities.

  Use Laplace (add-1) smoothing.
  Return a nested dictionary. The outer dictionary needs map terms to the 
  inner dictionaries. The inner dictionary maps class labels to probabilities.

  -> If the dictionary is accessed with a non-existing key but an existing class
  it should return the value 1/|V| ! <--

  Example:
    dct = {"fruit": {"apple": 0.3, "banana": 0.4}, "tree": {"apple": 0.5, "banana": 0.1}}
    foo = dct["rock"]["apple"]
    foo == 1/2
    Two terms exist in total (fruit and tree) so the total vocabulary size is 2.

  HINT: Use defaultdict(): https://docs.python.org/3.6/library/collections.html#collections.defaultdict

  :param terms: A list of strings for classes to calculate the frequencies.
  :param freqs_per_class: A dictionary containing the frequency of terms appearing in texts of each class.
  :return: A nested dictionary contains the terms and classes with the term's appearance probability.
  
  """


  total_different_words = len(terms)

  classes = set()
  for class_ in freqs_per_class.keys():
    classes.add(class_)

  words_per_class = dict()
  for class_ in classes:
    words_per_class[class_] = sum(freqs_per_class[class_].values())
    
  #dct = defaultdict(lambda: defaultdict(lambda: 1/total_different_words))
  a = 1/total_different_words
  dct = defaultdict(lambda: {c: a for c in classes})
  
  for term in terms:
    dct[term] = dict()
    for class_ in classes:
      absolute_frequency = freqs_per_class[class_][term]
      words_in_class = words_per_class[class_]

      dct[term][class_] = (absolute_frequency + 1) / (words_in_class + total_different_words)


  

  return dct

In [49]:
############### EXTRA OUTPUT / TESTING ##################
if not grading:
  cond_probs = calculate_class_term_probs(terms, freqs_per_class)
  # correct: 0.00013765662333908876
  print(cond_probs["happy"]["pos"])
  # correct: 8.604442314225311e-05
  print(cond_probs["happy"]["neg"])
  # correct: 3.965232838472275e-06
  # NOTE: this is a word that does not exist in the data. therefore the value
  # becomes 1/len(terms) - this is what the formula above for laplace smoothing
  # tells us 
  print(cond_probs["pneumonoultramicroscopicsilicovolcanoconiosis"]["pos"])


0.00013765662333908876
8.604442314225311e-05
3.965232838472275e-06


In [0]:
if not grading:
  probs = calculate_class_term_probs(terms, freqs_per_class)

In [0]:
def classify(text, priors, probs):
  """
  TASK 1d)
  This function should predict a class for a given text.
  Use the priors you calculated in 1a).
  Use the probabilities you calculated in 1c).

  !!!!!!!
  Instead of multiplying the probabilites together you can take the logarithm
  of the probabilities and add them together. 
  !!!!!!!

  Use the Naive Bayes classification to assign the class.
  
  :param text: A string contains the text to clasiffy.
  :param priors: A dictionary mapping the class to its probability (From TASK 1a).
  :param probs: A nested dictionary contains the terms and classes with
  the term's appearance probability (From TASK 1c).
  :return: A string for the class label predicted.

  """

  classes = set()

  for class_ in priors:
    classes.add(class_)
  
  highestProb = []
  for class_ in classes:
    probability = 0
    for word in text.lower().split(" "):
      probability = probability + math.log(probs[word][class_])

    prob = math.log(priors[class_]) + probability

    if len(highestProb) == 0:
      highestProb = [class_, prob]
    elif prob > highestProb[1]:
      highestProb = [class_, prob]
    
  return highestProb[0]



In [53]:

############### EXTRA OUTPUT / TESTING ##################
if not grading:
  t1 = "Today is an awful day."
  pred = classify(t1, priors, probs)
  # neg
  print(f"'{t1}' is {pred}")
  t2 = "Today is a nice day."
  pred = classify(t2, priors, probs)
  # pos
  print(f"'{t2}' is {pred}")
  t3 = "pneumonoultramicroscopicsilicovolcanoconiosis"
  pred = classify(t3, priors, probs)
  # pos
  print(f"'{t3}' is {pred}")
  t4 = "This movie was very good and I liked it a lot."
  pred = classify(t4, priors, probs)
  # pos
  print(f"'{t4}' is {pred}")

'Today is an awful day.' is neg
'Today is a nice day.' is pos
'pneumonoultramicroscopicsilicovolcanoconiosis' is pos
'This movie was very good and I liked it a lot.' is pos


In [0]:
def evaluate(test, priors, probs):
  """
  TASK 1e)
  Evaluate your classification.

  -> You may assume that the only classes are "pos" and "neg" ! <--

  The function takes a DataFrame to evaluate the classifier from TASK 1d.
  For each row in the test dataset, get the actual class from the "class" column
  and the text from the "text" column.
  Use your "classify(text, priors, probs)" function to get the predicted
  class label for each text.

  HINT: Count the TP, FP, FN for each of the two classes, for each prediction. 

  Calculate the following metrics PER CLASS:
    - precision
    - recall
    - f1-measure

  :param test: A pandas.DataFrame containing a column "class" and a column "text".
  :param priors: A dictionary mapping the class to its probability (From TASK 1a).
  :param probs: A nested dictionary contains the terms and classes with
  the term's appearance probability (From TASK 1c).
  :return: A list of tuples in the following format:
    [(class(string), precision(float), recall(float), f1-measure(float)), ...]

  """

  results = list()

  def perf_measure(y_actual, y_hat, positive, negative):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==positive:
           TP += 1
        if y_hat[i]==positive and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==negative:
           TN += 1
        if y_hat[i]==negative and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

  real_value = list()
  predictions = list()
  for i, row in test.iterrows():
      real_value.append(row['class'])
      predictions.append(classify(row['text'], priors, probs))

  #Wrong values (does it even make sense to calculate per class!!!!!!!)
  tp, fp, tn, fn = perf_measure(real_value, predictions, 'pos', 'neg')
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1_measure = 2*(precision * recall)/(precision + recall)
  results.append(('pos', precision, recall, f1_measure))

  tp, fp, tn, fn = perf_measure(real_value, predictions, 'neg', 'pos')
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1_measure = 2*(precision * recall)/(precision + recall)
  results.append(('neg', precision, recall, f1_measure))

  return results

In [0]:
if not grading:
  results = evaluate(test_df, priors, probs)

In [0]:
############### EXTRA OUTPUT / TESTING ##################
if not grading:
  results = evaluate(test_df, priors, probs)
  for cls, p, r, f1 in results:
    print(cls, p, r, f1)
  # pos 0.8594459784448205 0.77192 0.8133350191764657
  # neg 0.7930007986640528 0.87376 0.8314238952536825

#Does not give right output???!!!

pos 0.8594459784448205 0.77192 0.8133350191764657
neg 0.7930007986640528 0.87376 0.8314238952536825
